In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_log_error as rmsle
from sklearn.metrics import root_mean_squared_error
import xgboost as xgb

In [7]:
#data set from kaggle: https://www.kaggle.com/competitions/grupo-bimbo-inventory-demand/data

#office
train = pd.read_csv("train.csv", usecols=['Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Producto_ID', 'Cliente_ID', 'Demanda_uni_equil'])
test = pd.read_csv("test.csv", usecols=['Semana', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK', 'Producto_ID', 'Cliente_ID', 'id'])

train = train.rename(columns={'Semana': 'Week_num',
                              'Agencia_ID': 'Sales_Depot_ID',
                              'Canal_ID': 'Sales_Channel_ID',
                              'Ruta_SAK': 'Route_ID',
                              'Cliente_ID': 'Client_ID',
                              'Venta_uni_hoy': 'Sales_unit_this_week',
                              'Venta_hoy': 'Sales_this_week',
                              'Dev_uni_proxima': 'Returns_unit_next_week',
                              'Dev_proxima': 'Returns_next_week',
                              'Demanda_uni_equil': 'adjusted_demand',
                              'Producto_ID': 'Product_ID'})

test = test.rename(columns={'Semana': 'Week_num',
                            'Agencia_ID': 'Sales_Depot_ID',
                            'Canal_ID': 'Sales_Channel_ID',
                            'Ruta_SAK': 'Route_ID',
                            'Cliente_ID': 'Client_ID',
                            'Venta_uni_hoy': 'Sales_unit_this_week',
                            'Venta_hoy': 'Sales_this_week',
                            'Dev_uni_proxima': 'Returns_unit_next_week',
                            'Dev_proxima': 'Returns_next_week',
                            'Demanda_uni_equil': 'adjusted_demand',
                            'Producto_ID': 'Product_ID'})



#set a unique id for each sales depot id, sales channel id, route id, client, product combination (thanks Gemini)
combined_df = pd.concat([train,test])
combined_df['ID'] = combined_df.groupby(['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Client_ID', 'Product_ID']).ngroup()

#set a combined client ID, consisting of a unique sales depot ID, sales channel ID, route ID, and client ID
combined_df['ccid'] = combined_df.groupby(['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Client_ID']).ngroup()

#set a combined product ID, consisting of a unique sales depot ID, sales channel ID, route ID, and product ID
combined_df['cpid'] = combined_df.groupby(['Sales_Depot_ID', 'Sales_Channel_ID', 'Route_ID', 'Product_ID']).ngroup()

train = combined_df.iloc[:len(train)].copy()
test = combined_df.iloc[len(train):].copy()

del combined_df


train = train.drop(columns='id')
train['adjusted_demand'] = train['adjusted_demand'].astype(int)
train['adjusted_demand'] = np.log1p(train['adjusted_demand'])
train = train.sort_values(by=['ID', 'Week_num']).reset_index(drop=True)

test = test.drop(columns='adjusted_demand')
test['id'] = test['id'].astype(int)
test = test.sort_values(by=['ID', 'Week_num']).reset_index(drop=True)

In [8]:
#create a dataframe of aggregate statistics for each client
testagg = train[train['Week_num'] <= 8].sort_values(by=['ccid']).groupby(['ccid'], as_index=False).agg({'Product_ID':'nunique', 'adjusted_demand':['mean', 'median', 'min', 'max']})

client_stats = pd.DataFrame()

client_stats['ccid'] = testagg['ccid']
client_stats['Products'] = testagg['Product_ID']['nunique']
client_stats['adj_dem_mean'] = testagg['adjusted_demand']['mean'].round(2)
client_stats['adj_dem_median'] = testagg['adjusted_demand']['median'].astype(int)
client_stats['adj_dem_min'] = testagg['adjusted_demand']['min']
client_stats['adj_dem_max'] = testagg['adjusted_demand']['max']

del testagg

#create a dataframe of aggregate statistics for each product
testagg = train[train['Week_num'] <= 8].sort_values(by=['cpid']).groupby(['cpid'], as_index=False).agg({'Client_ID':'nunique', 'adjusted_demand':['mean', 'median', 'min', 'max']})

product_stats =  pd.DataFrame()

product_stats['cpid'] = testagg['cpid']
product_stats['Clients'] = testagg['Client_ID']['nunique']
product_stats['adj_dem_mean'] = testagg['adjusted_demand']['mean'].round(2)
product_stats['adj_dem_median'] = testagg['adjusted_demand']['median'].astype(int)
product_stats['adj_dem_min'] = testagg['adjusted_demand']['min']
product_stats['adj_dem_max'] = testagg['adjusted_demand']['max']
product_stats['median_pct'] = product_stats['adj_dem_median'].rank(pct=True, method='average')

del testagg

In [11]:
# get ccid, cpid means, medians and cpid median percentage into training data
cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())].set_index('ccid')['adj_dem_mean'], index=client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())]['ccid']).to_dict()
train['ccid_mean'] = train['ccid'].map(cidmapping)

cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())].set_index('ccid')['adj_dem_median'], index=client_stats[client_stats['ccid'].isin(train['ccid'].unique().tolist())]['ccid']).to_dict()
train['ccid_median'] = train['ccid'].map(cidmapping)

pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())].set_index('cpid')['adj_dem_mean'], index=product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())]['cpid']).to_dict()
train['cpid_mean'] = train['cpid'].map(pidmapping)

pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())].set_index('cpid')['adj_dem_median'], index=product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())]['cpid']).to_dict()
train['cpid_median'] = train['cpid'].map(pidmapping)

pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())].set_index('cpid')['median_pct'], index=product_stats[product_stats['cpid'].isin(train['cpid'].unique().tolist())]['cpid']).to_dict()
train['cpid_median_pct'] = train['cpid'].map(pidmapping).round(3)


# get total number of different products purchased by each client
# get total number of clients 

del cidmapping, pidmapping

In [10]:
# Define the fraction of IDs to sample
fraction = 0.2

# Calculate the number of IDs to sample
unique_ids = train['ID'].unique()
sample_size = int(len(unique_ids) * fraction)

# Choose a random sample of IDs
sampled_ids = np.random.choice(unique_ids, size=sample_size, replace=False)

# Filter the DataFrame to keep all rows with the sampled IDs
train = train[train['ID'].isin(sampled_ids)]

Impute NaN week 9 values by using the best simple prediction based on weeks 3-8.

In [48]:
wk8IDs = train[train['Week_num'] == 8]['ID'].unique().tolist()
wk9IDs = train[train['Week_num'] == 9]['ID'].unique().tolist()
wk8and9IDs = list(set(wk8IDs) & set(wk9IDs))

iw9 = train[['ID', 'ccid', 'ccid_mean', 'ccid_median', 'cpid', 'cpid_mean', 'cpid_median', 'cpid_median_pct']].drop_duplicates(subset='ID', keep='first').reset_index(drop=True).copy(deep=True)



# are the cpid, ccids in the training data?
iw9.loc[:, 'cpid_in_train'] = iw9['cpid'].isin(train[train['Week_num'] <= 8]['cpid'].unique().tolist())
iw9.loc[:, 'ccid_in_train'] = iw9['ccid'].isin(train[train['Week_num'] <= 8]['ccid'].unique().tolist())

# initialize adjusted demand column
iw9['adjusted_demand'] = np.zeros(len(iw9))

# for ccid and cpid not in training data, set adjusted demand to 5
iw9.loc[(iw9['cpid_in_train'] == False) & (iw9['ccid_in_train'] == False), 'adjusted_demand'] = np.log1p(5)

# for cpid in training data and ccid not in training data, use cpid median
iw9.loc[(iw9['ccid_in_train'] == False) & (iw9['cpid_in_train'] == True), 'adjusted_demand'] = iw9[(iw9['ccid_in_train'] == False) & (iw9['cpid_in_train'] == True)]['cpid_median']

# for ccid in training data and cpid not in training data, use ccid median
iw9.loc[(iw9['ccid_in_train'] == True) & (iw9['cpid_in_train'] == False), 'adjusted_demand'] = iw9[(iw9['ccid_in_train'] == True) & (iw9['cpid_in_train'] == False)]['ccid_median']

# for ccid, cpid in training data, use (0.5 + (cpid median pct)) * (0.65 * (ccid median) + (1-0.65) * (ccid mean))
iw9.loc[(iw9['ccid_in_train'] == True) & (iw9['cpid_in_train'] == True), 'adjusted_demand'] = (0.5 + iw9[(iw9['ccid_in_train'] == True) & (iw9['cpid_in_train'] == True)]['cpid_median_pct'])*(0.65*iw9[(iw9['ccid_in_train'] == True) & (iw9['cpid_in_train'] == True)]['ccid_median'] + (1-0.65)*iw9[(iw9['ccid_in_train'] == True) & (iw9['cpid_in_train'] == True)]['ccid_mean'])



# override with linear regression predicted adjusted demand for the IDs that have week 9 data


# only use data points (adjusted demand) if they appear in consecutive weeks, i.e. week 3 and week 4 or week 6 and week 7.
conseq_col = train['Week_num'].diff().dropna().astype(int)
conseq_col.loc[0] = 0
conseq_col = conseq_col.sort_index()

train['conseq_pts'] = conseq_col

train['adj_dem_lag1'] = train['adjusted_demand'].shift(1)

lr = LinearRegression()

lr.fit(X=train[(train['conseq_pts'] == 1) & (train['Week_num'] <= 8)][['adj_dem_lag1']].values, y=train[(train['conseq_pts'] == 1) & (train['Week_num'] <= 8)][['adjusted_demand']].values)

# get week 9 prediction
iw9.loc[iw9['ID'].isin(wk8and9IDs), 'adjusted_demand'] = lr.predict(train[(train['Week_num'] == 8) & (train['ID'].isin(train[train['Week_num'] == 9]['ID'].unique().tolist()))][['adjusted_demand']].values)

iw9

,ID,ccid,ccid_mean,ccid_median,cpid,cpid_mean,cpid_median,cpid_median_pct,cpid_in_train,ccid_in_train,adjusted_demand
0,1,0,1.72,1.0,5,0.99,1.0,0.399,True,True,1.125548
1,6,0,1.72,1.0,20,1.57,1.0,0.399,True,True,1.209305
2,17,0,1.72,1.0,39,1.18,1.0,0.399,True,True,1.125548
3,22,0,1.72,1.0,47,1.14,1.0,0.399,True,True,1.125548
4,24,0,1.72,1.0,50,1.75,1.0,0.399,True,True,1.125548
...,...,...,...,...,...,...,...,...,...,...,...
5279324,27752054,2285196,1.14,1.0,1131519,1.68,1.0,0.399,True,True,1.209305
5279325,27752057,2285196,1.14,1.0,1131527,1.76,1.0,0.399,True,True,0.943051
5279326,27752061,2285196,1.14,1.0,1131560,NaN,NaN,NaN,False,True,1.000000
5279327,27752089,2285202,1.72,1.0,1131541,1.53,1.0,0.399,True,True,1.125548


In [49]:
rmsle(np.expm1(train[train['Week_num'] == 9]['adjusted_demand']), np.expm1(iw9.loc[iw9['ID'].isin(wk9IDs), 'adjusted_demand']))

0.5902885740785169

In [45]:
# create training data based on ID in order to use lagged adjusted demand
trainIDdf = pd.DataFrame()

trainIDdf = train[['ID', 'Client_ID', 'Product_ID', 'ccid_mean', 'ccid_median', 'cpid_mean', 'cpid_median']].drop_duplicates(subset='ID', keep='first').reset_index(drop=True)

# get adjusted demand for the week for each ID 
for j in [6,7,8,9]:
    wkmap = pd.Series(train[train['Week_num'] == j].set_index('ID')['adjusted_demand'], index=train['ID'].unique()).to_dict()
    trainIDdf[f'Wk_{j}_dem'] = trainIDdf['ID'].map(wkmap)

trainIDdf['ID'] = trainIDdf['ID'].astype('category')
trainIDdf['Client_ID'] = trainIDdf['Client_ID'].astype('category')
trainIDdf['Product_ID'] = trainIDdf['Product_ID'].astype('category')

trainIDdf.head()

,ID,Client_ID,Product_ID,ccid_mean,ccid_median,cpid_mean,cpid_median,Wk_6_dem,Wk_7_dem,Wk_8_dem,Wk_9_dem
0,1,15766,328,1.72,1.0,0.99,1.0,NaN,NaN,NaN,NaN
1,6,15766,1240,1.72,1.0,1.57,1.0,NaN,2.197225,1.098612,1.098612
2,17,15766,5350,1.72,1.0,1.18,1.0,NaN,NaN,NaN,1.098612
3,22,15766,30551,1.72,1.0,1.14,1.0,NaN,NaN,NaN,NaN
4,24,15766,30574,1.72,1.0,1.75,1.0,1.94591,NaN,NaN,NaN


In [51]:
X_train = trainIDdf.iloc[:, 1:-1].copy(deep=True)

trainIDdf['Wk_9_dem'].fillna(iw9['adjusted_demand'], inplace=True)
y_train = trainIDdf.iloc[:, -1].copy(deep=True)

# X_train.drop(['Wk_3_dem', 'Wk_4_dem','Wk_5_dem'], axis=1, inplace=True)
X_train.rename(columns={'Wk_6_dem': 'lag_3', 'Wk_7_dem': 'lag_2', 'Wk_8_dem': 'lag_1'}, inplace=True)

X_train.head()

C:\Users\eblac\AppData\Local\Temp\ipykernel_16356\255397869.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  trainIDdf['Wk_9_dem'].fillna(iw9['adjusted_demand'], inplace=True)


,Client_ID,Product_ID,ccid_mean,ccid_median,cpid_mean,cpid_median,lag_3,lag_2,lag_1
0,15766,328,1.72,1.0,0.99,1.0,NaN,NaN,NaN
1,15766,1240,1.72,1.0,1.57,1.0,NaN,2.197225,1.098612
2,15766,5350,1.72,1.0,1.18,1.0,NaN,NaN,NaN
3,15766,30551,1.72,1.0,1.14,1.0,NaN,NaN,NaN
4,15766,30574,1.72,1.0,1.75,1.0,1.94591,NaN,NaN


In [52]:
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)

In [53]:
model1 = xgb.train({'max_depth': 8, 'eta': 0.1, 'objective': 'reg:squaredlogerror', 'tree_method': 'gpu_hist'}, dtrain, num_boost_round=25)
model2 = xgb.train({'max_depth': 8, 'eta': 0.1, 'objective': 'reg:squaredlogerror', 'tree_method': 'gpu_hist'}, dtrain, num_boost_round=50)
model3 = xgb.train({'max_depth': 8, 'eta': 0.1, 'objective': 'reg:squaredlogerror', 'tree_method': 'gpu_hist'}, dtrain, num_boost_round=75)
model4 = xgb.train({'max_depth': 8, 'eta': 0.1, 'objective': 'reg:squaredlogerror', 'tree_method': 'gpu_hist'}, dtrain, num_boost_round=100)

c:\Users\eblac\anaconda3\envs\erdos_spring_2025\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:19:38] WARNING: D:\bld\xgboost-split_1737698232980\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\eblac\anaconda3\envs\erdos_spring_2025\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:20:48] WARNING: D:\bld\xgboost-split_1737698232980\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
c:\Users\eblac\anaconda3\envs\erdos_spring_2025\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:24:02] WARNING: D:\bld\xgboost-split_1737698232980\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is 

In [54]:
test_pred = model1.predict(xgb.DMatrix(X_train, enable_categorical=True))
test_pred[test_pred < 0] = 0

print("model1:", rmsle(np.expm1(test_pred), np.expm1(y_train)))

test_pred = model2.predict(xgb.DMatrix(X_train, enable_categorical=True))
test_pred[test_pred < 0] = 0

print("model2:", rmsle(np.expm1(test_pred), np.expm1(y_train)))

test_pred = model3.predict(xgb.DMatrix(X_train, enable_categorical=True))
test_pred[test_pred < 0] = 0

print("model3:", rmsle(np.expm1(test_pred), np.expm1(y_train)))

test_pred = model4.predict(xgb.DMatrix(X_train, enable_categorical=True))
test_pred[test_pred < 0] = 0

print("model4:", rmsle(np.expm1(test_pred), np.expm1(y_train)))

c:\Users\eblac\anaconda3\envs\erdos_spring_2025\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:35:43] WARNING: D:\bld\xgboost-split_1737698232980\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


model1: 0.43978223552111045


c:\Users\eblac\anaconda3\envs\erdos_spring_2025\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:35:44] WARNING: D:\bld\xgboost-split_1737698232980\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


model2: 0.3529093346430481


c:\Users\eblac\anaconda3\envs\erdos_spring_2025\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:35:45] WARNING: D:\bld\xgboost-split_1737698232980\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


model3: 0.34863421523396765


c:\Users\eblac\anaconda3\envs\erdos_spring_2025\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:35:46] WARNING: D:\bld\xgboost-split_1737698232980\work\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


model4: 0.34780341797203246


In [8]:
model1 = xgb.train({'max_depth': 4, 'eta': 0.1, 'objective': 'reg:squaredlogerror'}, dtrain, num_boost_round=50)
model2 = xgb.train({'max_depth': 6, 'eta': 0.1, 'objective': 'reg:squaredlogerror'}, dtrain, num_boost_round=50)
model3 = xgb.train({'max_depth': 8, 'eta': 0.1, 'objective': 'reg:squaredlogerror'}, dtrain, num_boost_round=50)
model4 = xgb.train({'max_depth': 10, 'eta': 0.1, 'objective': 'reg:squaredlogerror'}, dtrain, num_boost_round=50)

In [9]:
test_pred = model1.predict(xgb.DMatrix(X_train, enable_categorical=True))
test_pred[test_pred < 0] = 0

print("model1:", rmsle(np.expm1(test_pred), np.expm1(y_train)))

test_pred = model2.predict(xgb.DMatrix(X_train, enable_categorical=True))
test_pred[test_pred < 0] = 0

print("model2:", rmsle(np.expm1(test_pred), np.expm1(y_train)))

test_pred = model3.predict(xgb.DMatrix(X_train, enable_categorical=True))
test_pred[test_pred < 0] = 0

print("model3:", rmsle(np.expm1(test_pred), np.expm1(y_train)))

test_pred = model4.predict(xgb.DMatrix(X_train, enable_categorical=True))
test_pred[test_pred < 0] = 0

print("model4:", rmsle(np.expm1(test_pred), np.expm1(y_train)))

model1: 0.3685791404244987
model2: 0.36017671060316475
model3: 0.3543582304155013
model4: 0.3497146206117892


In [55]:
# get ccid, cpid means and medians into test data
cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(test['ccid'].unique().tolist())].set_index('ccid')['adj_dem_mean'], index=client_stats[client_stats['ccid'].isin(test['ccid'].unique().tolist())]['ccid']).to_dict()
test['ccid_mean'] = test['ccid'].map(cidmapping)

cidmapping = pd.Series(client_stats[client_stats['ccid'].isin(test['ccid'].unique().tolist())].set_index('ccid')['adj_dem_median'], index=client_stats[client_stats['ccid'].isin(test['ccid'].unique().tolist())]['ccid']).to_dict()
test['ccid_median'] = test['ccid'].map(cidmapping)

pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(test['cpid'].unique().tolist())].set_index('cpid')['adj_dem_mean'], index=product_stats[product_stats['cpid'].isin(test['cpid'].unique().tolist())]['cpid']).to_dict()
test['cpid_mean'] = test['cpid'].map(pidmapping)

pidmapping = pd.Series(product_stats[product_stats['cpid'].isin(test['cpid'].unique().tolist())].set_index('cpid')['adj_dem_median'], index=product_stats[product_stats['cpid'].isin(test['cpid'].unique().tolist())]['cpid']).to_dict()
test['cpid_median'] = test['cpid'].map(pidmapping)

del cidmapping, pidmapping

test = test[['id', 'ID', 'Week_num', 'Client_ID', 'Product_ID', 'ccid_mean', 'ccid_median', 'cpid_mean', 'cpid_median']].sort_values(by='id')


# get adjusted demand from previous weeks
lagmap = pd.Series(trainIDdf[['ID', 'Wk_7_dem']].set_index('ID')['Wk_7_dem'], index=trainIDdf['ID'].tolist()).to_dict()
test['lag_3'] = test['ID'].map(lagmap)

lagmap = pd.Series(trainIDdf[['ID', 'Wk_8_dem']].set_index('ID')['Wk_8_dem'], index=trainIDdf['ID'].tolist()).to_dict()
test['lag_2'] = test['ID'].map(lagmap)

lagmap = pd.Series(trainIDdf[['ID', 'Wk_9_dem']].set_index('ID')['Wk_9_dem'], index=trainIDdf['ID'].tolist()).to_dict()
test['lag_1'] = test['ID'].map(lagmap)

del lagmap


test['Client_ID'] = test['Client_ID'].astype('category')
test['Product_ID'] = test['Product_ID'].astype('category')

test.head()

,id,ID,Week_num,Client_ID,Product_ID,ccid_mean,ccid_median,cpid_mean,cpid_median,lag_3,lag_2,lag_1
6558101,0,25973294,11,4639078,35305,1.21,1.0,1.67,1.0,NaN,NaN,NaN
5991443,1,23662849,11,4705135,1238,1.32,1.0,1.03,1.0,NaN,NaN,NaN
5398738,2,21257171,10,4549769,32940,1.73,1.0,1.16,1.0,NaN,1.098612,1.098612
1383046,3,5334985,11,4717855,43066,1.05,0.0,0.78,0.0,NaN,NaN,NaN
1110570,4,4150753,11,966351,1277,1.68,1.0,NaN,NaN,NaN,NaN,NaN


In [56]:
# predict week 10 data
X_test_wk10 = xgb.DMatrix(test[test['Week_num'] == 10].iloc[:, 3:], enable_categorical=True)

predictions_10 = model4.predict(X_test_wk10)


# add week 10 prediction to test
test['wk_10_pred_dem'] = np.nan
test.loc[test['Week_num'] == 10, 'wk_10_pred_dem'] = predictions_10


# predict week 11 data
test_wk11 = test[test['Week_num'] == 11].copy(deep=True)

test_wk11.drop(['lag_3'], axis=1, inplace=True)
test_wk11.rename(columns={'lag_2': 'lag_3', 'lag_1': 'lag_2', 'wk_10_pred_dem': 'lag_1'}, inplace=True)

X_test_wk11 = xgb.DMatrix(test_wk11.iloc[:, 3:], enable_categorical=True)

predictions_11 = model4.predict(X_test_wk11)


# add week 11 prediction to test
test['wk_11_pred_dem'] = np.nan
test.loc[test['Week_num'] == 11, 'wk_11_pred_dem'] = predictions_11


test.reset_index(drop=True, inplace=True)
test.head()

,id,ID,Week_num,Client_ID,Product_ID,ccid_mean,ccid_median,cpid_mean,cpid_median,lag_3,lag_2,lag_1,wk_10_pred_dem,wk_11_pred_dem
0,0,25973294,11,4639078,35305,1.21,1.0,1.67,1.0,NaN,NaN,NaN,NaN,1.003540
1,1,23662849,11,4705135,1238,1.32,1.0,1.03,1.0,NaN,NaN,NaN,NaN,0.969618
2,2,21257171,10,4549769,32940,1.73,1.0,1.16,1.0,NaN,1.098612,1.098612,1.11381,NaN
3,3,5334985,11,4717855,43066,1.05,0.0,0.78,0.0,NaN,NaN,NaN,NaN,0.294357
4,4,4150753,11,966351,1277,1.68,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.304518


In [57]:
submission = pd.DataFrame()

submission['id'] = np.arange(len(test))
submission['Demanda_uni_equil'] = test['wk_10_pred_dem'].combine_first(test['wk_11_pred_dem'])
submission['Demanda_uni_equil'] = np.expm1(submission['Demanda_uni_equil'])
submission.loc[submission['Demanda_uni_equil'] < 0, 'Demanda_uni_equil'] = 0

submission.head()

,id,Demanda_uni_equil
0,0,1.727920
1,1,1.636937
2,2,2.045942
3,3,0.342263
4,4,2.685913


In [58]:
submission.to_csv("xgb_prediction_2_log_ss.csv", index=False)